In [130]:
import pandas as pd
import csv
import geopandas as gpd

In [131]:
population = pd.read_csv('../data/raw/population_for_suburb.csv',index_col=0)
population_pred_greater_melbourne= pd.read_csv('../data/raw/population_pred_greater_melbourne.csv',index_col=0)
population_pred_rest_vic= pd.read_csv('../data/raw/population_pred_rest_vic.csv',index_col=0)
population_pred_vic= pd.read_csv('../data/raw/population_pred_vic.csv',index_col=0)
properties_preporcessed_1=pd.read_csv('../data/raw/properties_preporcessed_1.csv',index_col=0)

SA2_2021_AUST_GDA2020=pd.read_csv('../data/raw/SA2_2021_AUST_GDA2020.csv',index_col=0)


In [132]:
population_pred_greater_melbourne=population_pred_greater_melbourne.loc[2022:2024,['Annual population growth(e)']]
population_pred_rest_vic=population_pred_rest_vic.loc[2022:2024,['Annual population growth(e)']]
population_pred_vic=population_pred_vic.loc[2022:2024,['Annual population growth(e)']]


population_pred_vic


,Annual population growth(e)
2022,2.0
2023,2.0
2024,1.9


In [133]:

properties_preporcessed_1=properties_preporcessed_1[['SA2_CODE21','LOC_PID']].copy()
properties_preporcessed_1=properties_preporcessed_1.groupby(['SA2_CODE21','LOC_PID']).count()


properties_preporcessed_1=properties_preporcessed_1.reset_index()
properties_preporcessed_1.SA2_CODE21=properties_preporcessed_1.SA2_CODE21.astype('int')
# properties_preporcessed_1=properties_preporcessed_1.set_index('SA2_CODE21',verify_integrity=True)
properties_preporcessed_1

,SA2_CODE21,LOC_PID
0,201011001,loc82f5b45a790d
1,201011001,loc8c852c59ffe5
2,201011001,loc98325a7e67bf
3,201011002,loc2c1864e3cbb6
4,201011002,loc76166a14c514
...,...,...
892,217041478,loce829f646f7ca
893,217041478,locf00a01fc543b
894,217041479,loc1f73672977ce
895,217041480,loc1f73672977ce


In [135]:
SA2_2021_AUST_GDA2020=SA2_2021_AUST_GDA2020[['SA2_CODE21','GCC_NAME21']]
SA2_2021_AUST_GDA2020=SA2_2021_AUST_GDA2020[SA2_2021_AUST_GDA2020['GCC_NAME21'].isin(['Greater Melbourne','Rest of Vic.','Migratory - Offshore - Shipping (Vic.)'])].copy()
SA2_2021_AUST_GDA2020=SA2_2021_AUST_GDA2020.set_index('SA2_CODE21').reset_index()
SA2_2021_AUST_GDA2020.SA2_CODE21=SA2_2021_AUST_GDA2020.SA2_CODE21.astype('int')
SA2_2021_AUST_GDA2020

,SA2_CODE21,GCC_NAME21
0,201011001,Rest of Vic.
1,201011002,Rest of Vic.
2,201011005,Rest of Vic.
3,201011006,Rest of Vic.
4,201011007,Rest of Vic.
...,...,...
518,217041477,Rest of Vic.
519,217041478,Rest of Vic.
520,217041479,Rest of Vic.
521,217041480,Rest of Vic.


In [136]:
df2=pd.merge(properties_preporcessed_1,SA2_2021_AUST_GDA2020)
df2=df2[['LOC_PID','GCC_NAME21']].copy()
df2

,LOC_PID,GCC_NAME21
0,loc82f5b45a790d,Rest of Vic.
1,loc8c852c59ffe5,Rest of Vic.
2,loc98325a7e67bf,Rest of Vic.
3,loc2c1864e3cbb6,Rest of Vic.
4,loc76166a14c514,Rest of Vic.
...,...,...
892,loce829f646f7ca,Rest of Vic.
893,locf00a01fc543b,Rest of Vic.
894,loc1f73672977ce,Rest of Vic.
895,loc1f73672977ce,Rest of Vic.


In [138]:
population=pd.merge(population,df2)[['LOC_PID','suburb_population','GCC_NAME21']]

Greater_Melbourne=population[population['GCC_NAME21'].isin(['Greater Melbourne'])].copy()
Rest_of_Vic=population[population['GCC_NAME21'].isin(['Rest of Vic.'])].copy()

population

,LOC_PID,suburb_population,GCC_NAME21
0,loc0067a4549ed1,4897.000000,Rest of Vic.
1,loc00a9769647d7,26158.000000,Greater Melbourne
2,loc00a9769647d7,26158.000000,Greater Melbourne
3,loc00a9769647d7,26158.000000,Greater Melbourne
4,loc00a9769647d7,26158.000000,Greater Melbourne
...,...,...,...
1432,locff6258c8ea42,9003.705882,Greater Melbourne
1433,locff62fb6a898a,22420.000000,Greater Melbourne
1434,locffa1c8993b70,5488.609375,Greater Melbourne
1435,locffb43e78ab10,2133.333333,Rest of Vic.


In [139]:
Greater_Melbourne['2022']=Greater_Melbourne['suburb_population']*population_pred_greater_melbourne.loc[2022]['Annual population growth(e)']
Greater_Melbourne['2023']=Greater_Melbourne['2022']*population_pred_greater_melbourne.loc[2023]['Annual population growth(e)']
Greater_Melbourne['2024']=Greater_Melbourne['2023']*population_pred_greater_melbourne.loc[2024]['Annual population growth(e)']
Greater_Melbourne

,LOC_PID,suburb_population,GCC_NAME21,2022,2023,2024
1,loc00a9769647d7,26158.000000,Greater Melbourne,60163.400000,132359.480000,277954.908000
2,loc00a9769647d7,26158.000000,Greater Melbourne,60163.400000,132359.480000,277954.908000
3,loc00a9769647d7,26158.000000,Greater Melbourne,60163.400000,132359.480000,277954.908000
4,loc00a9769647d7,26158.000000,Greater Melbourne,60163.400000,132359.480000,277954.908000
5,loc00d1503504f1,41928.000000,Greater Melbourne,96434.400000,212155.680000,445526.928000
...,...,...,...,...,...,...
1430,locff0c4d75067f,8020.000000,Greater Melbourne,18446.000000,40581.200000,85220.520000
1432,locff6258c8ea42,9003.705882,Greater Melbourne,20708.523529,45558.751765,95673.378706
1433,locff62fb6a898a,22420.000000,Greater Melbourne,51566.000000,113445.200000,238234.920000
1434,locffa1c8993b70,5488.609375,Greater Melbourne,12623.801562,27772.363438,58321.963219


In [140]:
Rest_of_Vic['2022']=Rest_of_Vic['suburb_population']*population_pred_rest_vic.loc[2022]['Annual population growth(e)']
Rest_of_Vic['2023']=Rest_of_Vic['2022']*population_pred_rest_vic.loc[2023]['Annual population growth(e)']
Rest_of_Vic['2024']=Rest_of_Vic['2023']*population_pred_rest_vic.loc[2024]['Annual population growth(e)']
Rest_of_Vic

,LOC_PID,suburb_population,GCC_NAME21,2022,2023,2024
0,loc0067a4549ed1,4897.000000,Rest of Vic.,5386.700000,5386.700000,5386.700000
9,loc00e6e39d335b,1057.555556,Rest of Vic.,1163.311111,1163.311111,1163.311111
19,loc0115c25c528c,1106.000000,Rest of Vic.,1216.600000,1216.600000,1216.600000
21,loc01636b23f63e,1046.083333,Rest of Vic.,1150.691667,1150.691667,1150.691667
22,loc01699d7adbaa,1793.333333,Rest of Vic.,1972.666667,1972.666667,1972.666667
...,...,...,...,...,...,...
1398,locfa471b99ea58,1680.000000,Rest of Vic.,1848.000000,1848.000000,1848.000000
1416,locfdb55e6f8522,2227.111111,Rest of Vic.,2449.822222,2449.822222,2449.822222
1418,locfe8a70072c86,3830.000000,Rest of Vic.,4213.000000,4213.000000,4213.000000
1431,locff58d0167065,10487.000000,Rest of Vic.,11535.700000,11535.700000,11535.700000


In [142]:
pred = pd.concat([Greater_Melbourne,Rest_of_Vic])

In [143]:
pred

,LOC_PID,suburb_population,GCC_NAME21,2022,2023,2024
1,loc00a9769647d7,26158.000000,Greater Melbourne,60163.400000,132359.480000,277954.908000
2,loc00a9769647d7,26158.000000,Greater Melbourne,60163.400000,132359.480000,277954.908000
3,loc00a9769647d7,26158.000000,Greater Melbourne,60163.400000,132359.480000,277954.908000
4,loc00a9769647d7,26158.000000,Greater Melbourne,60163.400000,132359.480000,277954.908000
5,loc00d1503504f1,41928.000000,Greater Melbourne,96434.400000,212155.680000,445526.928000
...,...,...,...,...,...,...
1398,locfa471b99ea58,1680.000000,Rest of Vic.,1848.000000,1848.000000,1848.000000
1416,locfdb55e6f8522,2227.111111,Rest of Vic.,2449.822222,2449.822222,2449.822222
1418,locfe8a70072c86,3830.000000,Rest of Vic.,4213.000000,4213.000000,4213.000000
1431,locff58d0167065,10487.000000,Rest of Vic.,11535.700000,11535.700000,11535.700000


In [144]:
pred.to_csv("../data/raw/population_projection.csv")